In [1]:
from dotenv import load_dotenv
import os

from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.language_models import BaseChatModel

from typing import Sequence, Callable, Any
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import HumanMessage
from langchain_core.messages import SystemMessage, trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph


load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

In [177]:
model_openai = ChatOpenAI(model="gpt-4o-mini", max_completion_tokens=100)
model_anthropic = ChatAnthropic(model="claude-3-haiku-20240307")


In [73]:
models = [model_openai, model_anthropic]

In [156]:
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    prompt_template: ChatPromptTemplate
    model_used: BaseChatModel
    language: str


In [75]:
from langchain.cache import InMemoryCache
import langchain

langchain.cache = InMemoryCache()
# Or for persistent caching:
# from langchain.cache import SQLiteCache
# langchain.cache = SQLiteCache(database_path=".langchain.db")

In [149]:
def trimmer(model: BaseChatModel) -> Callable[[MessagesState], list[BaseMessage]]:
    return trim_messages(
        max_tokens=50,
        strategy="last",
        token_counter=model,
        include_system=True,
        allow_partial=True,
        start_on="human",
)

In [187]:
def call_first_layer_model(state: State) -> dict[str, list[BaseMessage]]:
    trimmed_messages = trimmer(model=state["model_used"]).invoke(state["messages"])
    template = state["prompt_template"]
    model = state["model_used"]
    prompt = template.invoke(
        {"messages": trimmed_messages, "language": state.get("language", "English")}, config)
    response = model.invoke(prompt)
    return {"messages": [response]}

### Lawyer

In [178]:
prompt_template_lawyer = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are an experienced attorney with decades of expertise, known for your thorough understanding of legal frameworks and your ability to articulate precise, formal, and well-reasoned arguments. "
                "Your goal is to provide comprehensive, professional responses that reflect a deep knowledge of the law, ethical considerations, and relevant legal principles. "
                "When appropriate, reference specific statutes, regulations, case law, or precedents to support your response. For example, in U.S. law, you might cite Marbury v. Madison (1803) for judicial review, or relevant sections of the UCC for commercial disputes. "
                "Structure your response logically with clear headings and subheadings if the question involves multiple aspects. Avoid assumptions beyond the given information and seek clarification if necessary. "
                "Always communicate in {language}, maintaining a professional and respectful tone, ensuring accessibility for both legal experts and non-specialists. "
                "Conclude your response with a summary or actionable next steps if applicable."
            ),
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [189]:
def create_graph(
    schema: dict[str, Any], 
    function: Callable[[State], dict[str, list[BaseMessage]]]
):
    workflow = StateGraph(state_schema=schema)
    workflow.add_edge(START, "llm_call")
    workflow.add_node("llm_call", function)
    return workflow.compile(checkpointer=MemorySaver())


In [180]:
query = "Hola, me llamo Ana. Me gustaria saber que es la EU AI Act."
language = "Spanish"
config = {"configurable": {"thread_id": "aaa123"}}

input_messages = [HumanMessage(query)]
output_lawyer = create_graph(State, call_first_layer_model).invoke({
    "messages": input_messages, 
    "language": language,
    "prompt_template": prompt_template_lawyer,
    "model_used": model_openai,
    }, 
config)
print(output_lawyer["messages"][-1].content)

Hola Ana,

### Introducción a la EU AI Act

La **EU AI Act** (Reglamento sobre Inteligencia Artificial de la Unión Europea) es una propuesta de legislación presentada por la Comisión Europea con el objetivo de regular el desarrollo y uso de tecnologías de inteligencia artificial (IA) en la Unión Europea. Este marco jurídico busca garantizar que la IA sea utilizada de manera segura y ética, promoviendo la innovación y protegiendo los derechos fundamentales de los ciudadanos.

### Objetivos Principales




### Social Media

In [181]:
prompt_template_social = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are a seasoned social media strategist with extensive experience in creating highly engaging and platform-specific content across Instagram, TikTok, YouTube, LinkedIn, and other major platforms. "
                "Your expertise includes crafting strategies that drive audience growth, boost engagement, and optimize monetization opportunities. "
                "Provide practical, actionable advice tailored to the platform in focus. For example, on Instagram, you might recommend using carousel posts for storytelling, leveraging trending Reels audio, or optimizing posting times based on analytics. "
                "On TikTok, suggest ideas for viral trends, short-form video hooks, or creative hashtag usage. "
                "Your recommendations should reflect the latest features and trends, such as LinkedIn’s algorithm preferences for professional storytelling or YouTube Shorts for reaching broader audiences. "
                "When discussing analytics, explain how to track and interpret key metrics like engagement rates, click-through rates (CTR), or watch time, providing examples where relevant. "
                "Offer community-building strategies, such as fostering meaningful interactions in comment sections or collaborating with influencers, to strengthen brand loyalty. "
                "Communicate in {language}, maintaining an engaging and approachable tone suitable for both novice and experienced content creators. "
                "Ensure your response includes real-world examples, practical tips, and clear steps for implementation."
            ),
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


In [238]:
query = "Me gustaria saber el equivalente de la EU AI Act en USA"
language = "Spanish"
config = {"configurable": {"thread_id": "aaa123"}}

input_messages = [HumanMessage(query)]
output_social = create_graph(State, call_first_layer_model).invoke({
    "messages": input_messages, 
    "language": language,
    "prompt_template": prompt_template_social,
    "model_used": model_openai,
    }, 
config)
print(output_social["messages"][-1].content)

¡Hola, Ana! Es genial que estés interesado en el tema de la regulación de la inteligencia artificial. A continuación, te explico cómo se están desarrollando las normativas en Estados Unidos en comparación con el EU AI Act.

### Equivalente del EU AI Act en EE. UU.

1. **No hay una ley federal única**: A diferencia de la Unión Europea, donde se está trabajando en un marco único (el EU AI Act) para regular la inteligencia artificial, en Estados Unidos


### Summarizer

In [239]:
class SummarizerState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    prompt_template: ChatPromptTemplate
    model_used: BaseChatModel
    language: str
    output_experts: list[dict[str, list[BaseMessage]]]

In [241]:
def call_second_layer_model(state: SummarizerState) -> dict[str, list[BaseMessage]]:
    trimmed_messages = trimmer(model=state["model_used"]).invoke(state["messages"])
    template = state["prompt_template"]
    model = state["model_used"]
    output_experts = "".join([f" Expert {i+1}: {expert['messages'][-1].content}" for i, expert in enumerate(state["output_experts"])])

    prompt = template.invoke({
        "messages": trimmed_messages, 
        "language": state.get("language", "English"),
        "output_experts": output_experts
    }, config)
    response = model.invoke(prompt)

    output_summarizer = f"""The experts have provided the following information: {output_experts}
        The summary of the information is: {response}
    """
    return {"messages": [output_summarizer]}

In [242]:
prompt_template_summarizer = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            ("""
                You are an expert summarizer with exceptional skills in synthesizing and distilling information from diverse sources.
                Your task is to combine the key insights from the experts provided here: {output_experts}.
                Craft a clear, concise, and cohesive summary that integrates their perspectives effectively, respecting each expert's unique focus and intent.\n\n
                Specifications:
                   - Provide a concise synthesis that integrates all perspectives.\n
                   - Ensure the summary is engaging, easy to understand, and tailored to the intended audience.\n\n
                When summarizing:\n
                - Maintain a professional yet conversational tone, ensuring accessibility for readers with varying levels of expertise.\n
                - Use analogies, examples, or simplified explanations to enhance understanding when complex concepts are involved.\n
                - Format your response with bullet points, headings, or short paragraphs for readability.\n\n
                Your response must be in {language} and must align with the style and context appropriate to the provided audience.
            """),
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


In [243]:
query = "Sintetiza los inputs de los expertos"
language = "Spanish"
config = {"configurable": {"thread_id": "aaa124"}}

input_messages = [HumanMessage(query)]
output_summarizer = create_graph(SummarizerState, call_second_layer_model).invoke({
    "messages": input_messages, 
    "language": language,
    "prompt_template": prompt_template_summarizer,
    "model_used": model_openai,
    "output_experts":[output_lawyer, output_social]
    }, 
config)
print(output_summarizer["messages"][-1].content)


        The experts have provided the following information:  Expert 1: Hola Ana,

### Introducción a la EU AI Act

La **EU AI Act** (Reglamento sobre Inteligencia Artificial de la Unión Europea) es una propuesta de legislación presentada por la Comisión Europea con el objetivo de regular el desarrollo y uso de tecnologías de inteligencia artificial (IA) en la Unión Europea. Este marco jurídico busca garantizar que la IA sea utilizada de manera segura y ética, promoviendo la innovación y protegiendo los derechos fundamentales de los ciudadanos.

### Objetivos Principales

 Expert 2: ¡Hola, Ana! Es genial que estés interesado en el tema de la regulación de la inteligencia artificial. A continuación, te explico cómo se están desarrollando las normativas en Estados Unidos en comparación con el EU AI Act.

### Equivalente del EU AI Act en EE. UU.

1. **No hay una ley federal única**: A diferencia de la Unión Europea, donde se está trabajando en un marco único (el EU AI Act) para regular l

In [83]:
query = "Hola, me llamo Ana. Me gustaria saber que es la EU AI Act."
language = "Spanish"
config = {"configurable": {"thread_id": "abc456"}}

input_messages = [HumanMessage(query)]
output = await app_anthropic.ainvoke({"messages": input_messages, "language": language}, config)
print(output["messages"][-1].content)

Aquí te presento una explicación del EU AI Act (Ley de Inteligencia Artificial de la Unión Europea) en español:

El EU AI Act es una propuesta de regulación de la Unión Europea para abordar los riesgos y desafíos asociados con el desarrollo y uso de la inteligencia artificial (IA). Los principales puntos clave son:

1. Ámbito de aplicación: La ley abarca todas las aplicaciones de IA, tanto públicas como privadas, que se desarrollen o utilicen en la UE.

2. Clasificación de sistemas de IA: Se establece una clasificación de sistemas de IA en función del nivel de riesgo que conllevan:
   - IA de riesgo inaceptable (prohibida)
   - IA de alto riesgo (sujeta a requisitos obligatorios)
   - IA de bajo o mínimo riesgo (sólo requiere transparencia)

3. Requisitos para sistemas de IA de alto riesgo:
   - Evaluación de riesgos y mitigación
   - Documentación y trazabilidad
   - Supervisión humana
   - Robustez, precisión y seguridad

4. Gobernanza y supervisión: Se crea un nuevo organismo europe

### Conversation

In [113]:
from typing import Sequence, Tuple, List
from langchain_core.messages import AIMessage
from datetime import datetime

def clean_message(text: str) -> str:
    """Clean message content by removing ALL whitespace issues"""
    if not isinstance(text, str) or not text:
        return ""
    # Remove ALL problematic whitespace
    text = text.strip()  # Remove leading/trailing whitespace
    text = ' '.join(text.split())  # Replace multiple spaces with single space
    text = text.rstrip()  # Extra insurance against trailing whitespace
    return text

In [102]:
clean_message("     Hola, me llamo Ana. Me gustaria saber que es la EU AI Act.")

'Hola, me llamo Ana. Me gustaria saber que es la EU AI Act.'

In [132]:
async def enhanced_chain_conversation(
    query: str,
    language: str,
    app_openai,
    app_anthropic,
    num_turns: int = 2,
    max_tokens: int = 10000
) -> dict:
    
    conversation_history = []
    metrics = {"token_usage": 0, "cost": 0}
    
    async def process_model_response(
        app, 
        message_content: str, 
        expert_name: str
    ) -> str:
        print(f"\nProcessing {expert_name} response...")
        
        # Create a clean message
        clean_content = clean_message(message_content)
        
        # Different message structure for each model, now with max_tokens
        if expert_name == "Legal Expert":
            messages = {
                "messages": [HumanMessage(content=clean_content)], 
                "language": language,
                "max_tokens": max_tokens  # Use the max_tokens parameter
            }
        else:
            messages = {
                "messages": [
                    SystemMessage(content="You are a social media expert. Convert complex information into engaging, clear content."),
                    HumanMessage(content=clean_content)
                ],
                "language": language,
                "max_tokens": max_tokens  # Use the max_tokens parameter
            }
        
        try:
            output = await app.ainvoke(messages, config)
            response = clean_message(output["messages"][-1].content)
            print(f"{expert_name} responded: {response[:100]}...")
            
            conversation_history.append({
                "role": expert_name,
                "content": response,
                "timestamp": datetime.now().isoformat()
            })
            
            return response
            
        except Exception as e:
            print(f"Error in {expert_name} response: {str(e)}")
            raise
    
    try:
        current_query = clean_message(query)
        
        # Now use num_turns for multiple exchanges
        for turn in range(num_turns):
            print(f"\n=== Turn {turn + 1}/{num_turns} ===")
            
            # Legal Expert Analysis
            legal_response = process_model_response(
                app_openai,
                current_query,
                "Legal Expert"
            )
            
            # Social Media Expert Translation
            social_prompt = (
                "Transform this legal explanation into engaging social media content. "
                "Keep it clear and accurate, but make it more accessible and interesting "
                f"for a general audience: {legal_response}"
            )
            
            social_response = process_model_response(
                app_anthropic,
                social_prompt,
                "Social Media Expert"
            )
            
            # Update query for next turn if there are more turns
            if turn < num_turns - 1:
                current_query = (
                    "Based on the previous responses, please provide additional insights "
                    "or explore another aspect of the EU AI Act. "
                    f"Previous legal response: {legal_response} "
                    f"Previous social media response: {social_response}"
                )
        
        # Generate final summary
        summary_prompt = (
            "Please provide a brief, clear summary of all the key points discussed "
            "in our conversation about the EU AI Act."
        )
        
        summary = process_model_response(
            app_anthropic,
            summary_prompt,
            "Summary"
        )
        
        return {
            "original_query": query,
            "conversation": conversation_history,
            "summary": summary,
            "metrics": metrics,
            "language": language
        }
        
    except Exception as e:
        print(f"\n❌ Error during conversation: {str(e)}")
        return None

In [133]:
# Example usage with specific turns and token limit
result = enhanced_chain_conversation(
    query=query,
    language=language,
    app_openai=app_openai,
    app_anthropic=app_anthropic,
    num_turns=3,  # Will do 3 rounds of exchanges
    #max_tokens=800  # Limit response lengths
)
print(result)

<coroutine object enhanced_chain_conversation at 0x000002064BE7F120>


In [121]:
async def chain_conversation(
    query: str,
    language: str,
    app_openai,
    app_anthropic,
    num_turns: int,
) -> dict:
    
    conversation_history = []
        
    async def process_model_response(
        app, 
        message_content: str, 
        expert_name: str,
    ) -> str:
        print(f"\nProcessing {expert_name} response...")
        
        clean_content = clean_message(message_content)




        
        
        # Different message structure for each model
        if expert_name == "Legal Expert":
            messages = {
                "messages": [HumanMessage(content=clean_content)],
                "language": language
            }
        else:
            # For Anthropic, only include system message when specified
            message_list = []
            if include_system:
                message_list.append(
                    SystemMessage(content="You are a social media expert. Convert complex information into engaging, clear content.")
                )
            message_list.append(HumanMessage(content=clean_content))
            messages = {
                "messages": message_list,
                "language": language
            }
        
        try:
            output = await app.ainvoke(messages, config)
            response = clean_message(output["messages"][-1].content)
            print(f"{expert_name} responded: {response[:100]}...")
            
            conversation_history.append({
                "role": expert_name,
                "content": response,
                "timestamp": datetime.now().isoformat()
            })
            
            return response
            
        except Exception as e:
            print(f"Error in {expert_name} response: {str(e)}")
            raise
    
    try:
        current_query = clean_message(query)
        
        for turn in range(num_turns):
            print(f"\n=== Turn {turn + 1}/{num_turns} ===")
            
            # Legal Expert Analysis
            legal_response = await process_model_response(
                app_openai,
                current_query,
                "Legal Expert"
            )
            
            # Social Media Expert Translation
            # Only include system message in first turn
            social_prompt = (
                "Transform this legal explanation into engaging social media content "
                "while maintaining accuracy and using appropriate language. "
                f"Legal explanation: {legal_response}"
            )
            
            social_response = await process_model_response(
                app_anthropic,
                social_prompt,
                "Social Media Expert",
                include_system=(turn == 0)  # Only include system message in first turn
            )
            
            # Update query for next turn
            if turn < num_turns - 1:
                current_query = clean_message(
                    "Based on the previous responses, please provide additional insights "
                    "about a different aspect of the EU AI Act that hasn't been covered yet. "
                    f"Previous legal response: {legal_response}"
                )
        
        # Generate final summary without system message
        summary_prompt = clean_message(
            "Please provide a brief, clear summary of all the key points discussed "
            "about the EU AI Act in our conversation."
        )
        
        summary = await process_model_response(
            app_anthropic,
            summary_prompt,
            "Summary",
            include_system=False  # Don't include system message for summary
        )
        
        return {
            "original_query": query,
            "conversation": conversation_history,
            "summary": summary,
            "metrics": metrics,
            "language": language
        }
        
    except Exception as e:
        print(f"\n❌ Error during conversation: {str(e)}")
        return None

In [123]:
result = await enhanced_chain_conversation(
    query=query,
    language=language,
    app_openai=app_openai,
    app_anthropic=app_anthropic,
    num_turns=3
)
print(result)


=== Turn 1/3 ===

Processing Legal Expert response...
Legal Expert responded: Hola, Ana. La EU AI Act, conocida formalmente como la Ley de Inteligencia Artificial de la Unión Eur...

Processing Social Media Expert response...
Error in Social Media Expert response: Received multiple non-consecutive system messages.

❌ Error during conversation: Received multiple non-consecutive system messages.
None


In [124]:
async def enhanced_chain_conversation(
    query: str,
    language: str,
    app_openai,
    app_anthropic,
    num_turns: int = 2,
    max_tokens: int = 1000
) -> dict:
    
    conversation_history = []
    metrics = {"token_usage": 0, "cost": 0}
    
    async def process_model_response(
        app, 
        message_content: str, 
        expert_name: str
    ) -> str:
        print(f"\nProcessing {expert_name} response...")
        
        # Clean input message
        clean_content = clean_message(message_content)
        
        # Create message with cleaned content
        message = HumanMessage(content=clean_content)
        
        # Get response from model
        output = await app.ainvoke(
            {"messages": [message], "language": language},
            config
        )
        
        if not output or "messages" not in output or not output["messages"]:
            raise ValueError("No valid response received from model")
        
        # Clean the response content
        response = clean_message(output["messages"][-1].content)
        
        # Store in conversation history
        conversation_history.append({
            "role": expert_name,
            "content": response,
            "timestamp": datetime.now().isoformat()
        })
        
        return response
    
    try:
        # Clean initial query
        current_query = clean_message(query)
        
        for turn in range(num_turns):
            print(f"\n=== Turn {turn + 1}/{num_turns} ===")
            
            # Legal Expert Analysis
            legal_response = await process_model_response(
                app_openai,
                current_query,
                "Legal Expert"
            )
            
            # Social Media Expert Translation
            social_prompt = clean_message(
                f"You are a social media expert. Please transform this legal explanation into engaging social media content "
                f"while maintaining accuracy. Use {language} language. Legal explanation: {legal_response}"
            )
            
            social_response = await process_model_response(
                app_anthropic,
                social_prompt,
                "Social Media Expert"
            )
            
            # Update query for next turn if needed
            if turn < num_turns - 1:
                current_query = clean_message(
                    f"Based on these explanations, what additional aspects of the EU AI Act should we explore? "
                    f"Previous legal response: {legal_response} "
                    f"Previous social media response: {social_response}"
                )
        
        # Generate final summary
        summary_prompt = clean_message(
            f"Please provide a brief summary in {language} of the key points discussed about the EU AI Act in this conversation."
        )
        
        summary = await process_model_response(
            app_anthropic,
            summary_prompt,
            "Summary"
        )
        
        return {
            "original_query": query,
            "conversation": conversation_history,
            "summary": summary,
            "metrics": metrics,
            "language": language
        }
        
    except Exception as e:
        print(f"\n❌ Error during conversation: {str(e)}")
        return None

# Example usage
async def run_conversation():
    query = "¿Qué es la EU AI Act y cómo afectará a las empresas?"
    language = "Spanish"
    global config
    config = {"configurable": {"thread_id": "test123"}}
    
    print(f"\n📝 Query: {query}")
    print("\n🔄 Processing conversation...\n")
    
    result = await enhanced_chain_conversation(
        query=query,
        language=language,
        app_openai=app_openai,
        app_anthropic=app_anthropic,
        num_turns=3
    )
    
    if result:
        print("\n=== Conversation Summary ===\n")
        print(f"🎯 Original Query: {result['original_query']}\n")
        
        for entry in result['conversation']:
            print(f"\n👤 {entry['role']}:")
            print(f"{entry['content']}\n")
            print("-" * 80)
        
        print("\n📋 Final Summary:")
        print(f"{result['summary']}\n")
    else:
        print("❌ An error occurred during the conversation")

# Execute
await run_conversation()


📝 Query: ¿Qué es la EU AI Act y cómo afectará a las empresas?

🔄 Processing conversation...


=== Turn 1/3 ===

Processing Legal Expert response...

Processing Social Media Expert response...

❌ Error during conversation: Received multiple non-consecutive system messages.
❌ An error occurred during the conversation


In [134]:
def chain_conversation(
    query: str,
    language: str,
    app_openai,
    app_anthropic,
    num_turns: int = 2,
    max_tokens: int = 1000
) -> dict:
    
    conversation_history = []
    metrics = {"token_usage": 0, "cost": 0}
    
    def process_model_response(
        app, 
        message_content: str, 
        expert_name: str
    ) -> str:
        print(f"\nProcessing {expert_name} response...")
        
        # Clean input message
        clean_content = clean_message(message_content)
        message = HumanMessage(content=clean_content)
        
        try:
            # Use invoke instead of ainvoke
            output = app.invoke(
                {"messages": [message], "language": language},
                config
            )
            
            if not output or "messages" not in output or not output["messages"]:
                raise ValueError("No valid response received from model")
            
            response = clean_message(output["messages"][-1].content)
            print(f"{expert_name} responded: {response[:100]}...")
            
            conversation_history.append({
                "role": expert_name,
                "content": response,
                "timestamp": datetime.now().isoformat()
            })
            
            return response
            
        except Exception as e:
            print(f"Error in {expert_name} response: {str(e)}")
            raise
    
    try:
        current_query = clean_message(query)
        
        for turn in range(num_turns):
            print(f"\n=== Turn {turn + 1}/{num_turns} ===")
            
            # Legal Expert Analysis
            legal_response = process_model_response(
                app_openai,
                current_query,
                "Legal Expert"
            )
            
            # Social Media Expert Translation
            social_prompt = clean_message(
                f"You are a social media expert. Please transform this legal explanation into engaging social media content "
                f"while maintaining accuracy. Use {language} language. Legal explanation: {legal_response}"
            )
            
            social_response = process_model_response(
                app_anthropic,
                social_prompt,
                "Social Media Expert"
            )
            
            # Update query for next turn if needed
            if turn < num_turns - 1:
                current_query = clean_message(
                    f"Based on these explanations, what additional aspects of the EU AI Act should we explore? "
                    f"Previous legal response: {legal_response} "
                    f"Previous social media response: {social_response}"
                )
        
        # Generate final summary
        summary_prompt = clean_message(
            f"Please provide a brief summary in {language} of the key points discussed about the EU AI Act in this conversation."
        )
        
        summary = process_model_response(
            app_anthropic,
            summary_prompt,
            "Summary"
        )
        
        return {
            "original_query": query,
            "conversation": conversation_history,
            "summary": summary,
            "metrics": metrics,
            "language": language
        }
        
    except Exception as e:
        print(f"\n❌ Error during conversation: {str(e)}")
        return None

def run_conversation():
    query = "¿Qué es la EU AI Act y cómo afectará a las empresas?"
    language = "Spanish"
    global config
    config = {"configurable": {"thread_id": "test123"}}
    
    print(f"\n📝 Query: {query}")
    print("\n🔄 Processing conversation...\n")
    
    result = chain_conversation(
        query=query,
        language=language,
        app_openai=app_openai,
        app_anthropic=app_anthropic,
        num_turns=3
    )
    
    if result:
        print("\n=== Conversation Summary ===\n")
        print(f"🎯 Original Query: {result['original_query']}\n")
        
        for entry in result['conversation']:
            print(f"\n👤 {entry['role']}:")
            print(f"{entry['content']}\n")
            print("-" * 80)
        
        print("\n📋 Final Summary:")
        print(f"{result['summary']}\n")
    else:
        print("❌ An error occurred during the conversation")

# Execute
print("Starting the conversation...")
run_conversation()
print("Conversation completed!")

Starting the conversation...

📝 Query: ¿Qué es la EU AI Act y cómo afectará a las empresas?

🔄 Processing conversation...


=== Turn 1/3 ===

Processing Legal Expert response...
Legal Expert responded: La EU AI Act, formalmente conocida como el Reglamento sobre la Inteligencia Artificial de la Unión E...

Processing Social Media Expert response...
Error in Social Media Expert response: Received multiple non-consecutive system messages.

❌ Error during conversation: Received multiple non-consecutive system messages.
❌ An error occurred during the conversation
Conversation completed!


In [93]:

async def enhanced_chain_conversation(
    query: str,
    language: str,
    num_turns: int = 2,
    max_tokens: int = 1000
) -> dict:
    
    conversation_history = []
    metrics = {"token_usage": 0, "cost": 0}
    
    async def process_model_response(
        app, 
        message_content: str, 
        expert_name: str
    ) -> str:
        # Clean input message
        clean_content = clean_message(message_content)
        
        # Create message with cleaned content
        message = HumanMessage(content=clean_content)
        
        # Get response from model
        output = await app.ainvoke(
            {"messages": [message], "language": language},
            config
        )
        
        if not output or "messages" not in output or not output["messages"]:
            raise ValueError("No valid response received from model")
        
        # Clean the response content
        response = clean_message(output["messages"][-1].content)
        
        # Store in conversation history
        conversation_history.append({
            "role": expert_name,
            "content": response,
            "timestamp": datetime.now().isoformat()
        })
        
        return response
    
    try:
        # Clean initial query
        current_query = clean_message(query)
        
        for turn in range(num_turns):
            # Legal Expert Analysis
            legal_response = await process_model_response(
                app_openai,
                current_query,
                "Legal Expert"
            )
            
            # Social Media Expert Translation
            social_prompt = clean_message(
                f"Transform this legal explanation into engaging social media content while maintaining accuracy: {legal_response}"
            )
            
            social_response = await process_model_response(
                app_anthropic,
                social_prompt,
                "Social Media Expert"
            )
            
            # Update query for next turn if needed
            if turn < num_turns - 1:
                current_query = clean_message(
                    f"Based on these explanations, what additional insights would be helpful? Legal: {legal_response} Social: {social_response}"
                )
        
        # Generate final summary
        summary_prompt = clean_message(
            "Provide a brief summary of the key points discussed in this conversation."
        )
        
        summary = await process_model_response(
            app_anthropic,
            summary_prompt,
            "Summary"
        )
        
        return {
            "original_query": query,
            "conversation": conversation_history,
            "summary": summary,
            "metrics": metrics,
            "language": language
        }
        
    except Exception as e:
        print(f"Error during conversation: {str(e)}")
        return None

In [97]:
from datetime import datetime
from typing import Sequence, Tuple, List
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

def clean_message(text: str) -> str:
    """Clean message content by removing excessive whitespace"""
    if not text:
        return ""
    text = text.strip()
    text = ' '.join(text.split())
    return text

async def enhanced_chain_conversation(
    query: str,
    language: str,
    app_openai, 
    app_anthropic,
    config: dict,
    num_turns: int = 2,
    max_tokens: int = 1000,
   ) -> dict:
    
    conversation_history = []
    metrics = {"token_usage": 0, "cost": 0}
    
    async def process_model_response(
        app, 
        message_content: str, 
        expert_name: str
    ) -> str:
        # Create a clean message
        clean_content = clean_message(message_content)
        messages = {"messages": [HumanMessage(content=clean_content)], "language": language}
        
        # Get response from model
        output = await app.ainvoke(messages, config)
        
        if not output or "messages" not in output or not output["messages"]:
            raise ValueError("No valid response received from model")
            
        response = clean_message(output["messages"][-1].content)
        
        # Update metrics (if output contains token usage/cost information)
        if "token_usage" in output:
            metrics["token_usage"] += output["token_usage"]
        if "cost" in output:
            metrics["cost"] += output["cost"]
        
        # Store in conversation history
        conversation_history.append({
            "role": expert_name,
            "content": response,
            "timestamp": datetime.now().isoformat()
        })
        
        return response
    
    try:
        # Initial query
        current_query = query
        
        for turn in range(num_turns):
            # Legal Expert Analysis
            legal_response = await process_model_response(
                app_openai,
                current_query,
                "Legal Expert"
            )
            
            # Social Media Expert Translation
            social_prompt = clean_message(
                f"Transform this legal explanation into engaging, clear content "
                f"that would work well on social media while maintaining accuracy: {legal_response}"
            )
            
            social_response = await process_model_response(
                app_anthropic,
                social_prompt,
                "Social Media Expert"
            )
            
            # Update query for next turn if needed
            if turn < num_turns - 1:
                current_query = clean_message(
                    "Based on both explanations above, what additional insights "
                    f"or clarifications would be helpful? Legal response: {legal_response} "
                    f"Social media response: {social_response}"
                )
        
        # Generate final summary
        summary_prompt = clean_message(
            "Please provide a brief, clear summary of the key points discussed "
            "in our conversation about this topic."
        )
        
        summary = await process_model_response(
            app_anthropic,
            summary_prompt,
            "Summary"
        )
        
        return {
            "original_query": query,
            "conversation": conversation_history,
            "summary": summary,
            "metrics": metrics,
            "language": language
        }
        
    except Exception as e:
        print(f"Error during conversation: {str(e)}")
        return None

In [98]:
async def run_conversation():
    query = "¿Qué es la EU AI Act y cómo afectará a las empresas?"
    language = "Spanish"
    global config
    config = {"configurable": {"thread_id": "test123"}}
    
    print(f"\n📝 Query: {query}")
    print("\n🔄 Processing conversation...\n")
    
    result = await enhanced_chain_conversation(
        query=query,
        language=language,
        app_openai=app_openai,  # Pass the OpenAI app
        app_anthropic=app_anthropic  # Pass the Anthropic app
    )
    
    if result:
        print("\n=== Conversation Summary ===\n")
        print(f"🎯 Original Query: {result['original_query']}\n")
        
        for entry in result['conversation']:
            print(f"\n👤 {entry['role']}:")
            print(f"{entry['content']}\n")
            print("-" * 80)
        
        print("\n📋 Final Summary:")
        print(f"{result['summary']}\n")
    else:
        print("❌ An error occurred during the conversation")

In [92]:
# Example usage with error handling and pretty printing
async def run_conversation():
    query = "¿Qué es la EU AI Act y cómo afectará a las empresas?"
    language = "Spanish"
    global config  # Make sure config is accessible
    config = {"configurable": {"thread_id": "test123"}}
    
    print(f"\n📝 Query: {query}")
    print("\n🔄 Processing conversation...\n")
    
    result = await enhanced_chain_conversation(query, language)
    
    if result:
        print("\n=== Conversation Summary ===\n")
        print(f"🎯 Original Query: {result['original_query']}\n")
        
        for entry in result['conversation']:
            print(f"\n👤 {entry['role']}:")
            print(f"{entry['content']}\n")
            print("-" * 80)
        
        print("\n📋 Final Summary:")
        print(f"{result['summary']}\n")
    else:
        print("❌ An error occurred during the conversation")

# Run the conversation
await run_conversation()


📝 Query: ¿Qué es la EU AI Act y cómo afectará a las empresas?

🔄 Processing conversation...

Error during conversation: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'messages: at least one message is required'}}
❌ An error occurred during the conversation


In [131]:
async def enhanced_chain_conversation(
    query: str,
    language: str,
    app_openai,
    app_anthropic,
    num_turns: int = 2,
    max_tokens: int = 1000
) -> dict:
    
    conversation_history = []
    metrics = {"token_usage": 0, "cost": 0}
    
    async def process_model_response(
        app, 
        message_content: str, 
        expert_name: str
    ) -> str:
        print(f"\nProcessing {expert_name} response...")
        
        # Create a clean message
        clean_content = clean_message(message_content)
        
        # Different message structure for each model
        if expert_name == "Legal Expert":
            messages = {"messages": [HumanMessage(content=clean_content)], "language": language}
        else:
            # For Anthropic, include a system message
            messages = {
                "messages": [
                    SystemMessage(content="You are a social media expert. Convert complex information into engaging, clear content."),
                    HumanMessage(content=clean_content)
                ],
                "language": language
            }
        
        try:
            output = await app.ainvoke(messages, config)
            
            if not output or "messages" not in output or not output["messages"]:
                raise ValueError("No valid response received from model")
            
            response = clean_message(output["messages"][-1].content)
            print(f"{expert_name} responded: {response[:100]}...")
            
            conversation_history.append({
                "role": expert_name,
                "content": response,
                "timestamp": datetime.now().isoformat()
            })
            
            return response
            
        except Exception as e:
            print(f"Error in {expert_name} response: {str(e)}")
            raise
    
    try:
        current_query = clean_message(query)
        
        # Legal Expert Analysis
        legal_response = await process_model_response(
            app_openai,
            current_query,
            "Legal Expert"
        )
        
        # Social Media Expert Translation
        social_prompt = (
            "Transform this legal explanation into engaging social media content. "
            "Keep it clear and accurate, but make it more accessible and interesting "
            f"for a general audience: {legal_response}"
        )
        
        social_response = await process_model_response(
            app_anthropic,
            social_prompt,
            "Social Media Expert"
        )
        
        # Generate final summary
        summary_prompt = (
            "Please provide a brief, clear summary of the key points discussed "
            "in our conversation about the EU AI Act."
        )
        
        summary = await process_model_response(
            app_anthropic,
            summary_prompt,
            "Summary"
        )
        
        return {
            "original_query": query,
            "conversation": conversation_history,
            "summary": summary,
            "metrics": metrics,
            "language": language
        }
        
    except Exception as e:
        print(f"\n❌ Error during conversation: {str(e)}")
        return None

# Run the conversation
async def run_conversation():
    query = "¿Qué es la EU AI Act y cómo afectará a las empresas?"
    language = "Spanish"
    global config
    config = {"configurable": {"thread_id": "test123"}}
    
    print(f"\n📝 Query: {query}")
    print("\n🔄 Processing conversation...\n")
    
    result = await enhanced_chain_conversation(
        query=query,
        language=language,
        app_openai=app_openai,
        app_anthropic=app_anthropic
    )
    
    if result:
        print("\n=== Conversation Summary ===\n")
        print(f"🎯 Original Query: {result['original_query']}\n")
        
        for entry in result['conversation']:
            print(f"\n👤 {entry['role']}:")
            print(f"{entry['content']}\n")
            print("-" * 80)
        
        print("\n📋 Final Summary:")
        print(f"{result['summary']}\n")
    else:
        print("❌ An error occurred during the conversation")

# Execute
print("Starting the conversation...")
await run_conversation()
print("Conversation completed!")

Starting the conversation...

📝 Query: ¿Qué es la EU AI Act y cómo afectará a las empresas?

🔄 Processing conversation...


Processing Legal Expert response...
Legal Expert responded: La EU AI Act, que se refiere a la propuesta de Ley de Inteligencia Artificial de la Unión Europea, e...

Processing Social Media Expert response...
Social Media Expert responded: ¡Hola! Hoy vamos a hablar de un tema importante y emocionante: ¡La Ley de Inteligencia Artificial de...

Processing Summary response...
Error in Summary response: Received multiple non-consecutive system messages.

❌ Error during conversation: Received multiple non-consecutive system messages.
❌ An error occurred during the conversation
Conversation completed!
